# Import

In [36]:
import os
import pandas as pd

import sagemaker
import boto3

from sagemaker.sklearn.estimator import SKLearn

from secret.config import ROLE

# SageMaker Setting

In [27]:
AWS_ACCESS_KEYS = pd.read_csv(os.path.join(os.getcwd(), "secret", "AccessKeys.csv"))

In [ ]:
os.environ["AWS_DEFAULT_REGION"] = 'ap-southeast-2'

boto_session = boto3.Session(
    aws_access_key_id = AWS_ACCESS_KEYS.loc[0,'Access key ID'],
    aws_secret_access_key = AWS_ACCESS_KEYS.loc[0,'Secret access key'],
    region_name = os.environ['AWS_DEFAULT_REGION']
)
sagemaker_session = sagemaker.local.LocalSession(
    boto_session=boto_session,
    default_bucket="mlflow-artifacts-dir"
)

# Define & Train Estimator

In [ ]:
ESTIMATOR_CFG = {
    'entry_point' : "./script/train_sklearn.py",
    'role' : ROLE,
    'framework_version' : '1.2-1',
    'py_version' : 'py3',
    'instance_type' : 'local',
    'instance_count' : 1,
    'output_path' : 's3://mlflow-tracking-dir/',
    'sagemaker_session' : sagemaker_session
}

PARAMETERS = {
    'n_estimators' : 200,
    'max_depth' : 7,
    'max_features' : 5,
    'experiment_name' : 'sklearn-estimator-local-lab',
    'run_name' : 'rf-clf-exp-1'
}

In [33]:
class RandomForestCLFEstimator:
    def __init__(self, estimator_cfg, param_set):
        self.estimator = SKLearn(
            entry_point=estimator_cfg['entry_point'],
            role=estimator_cfg['role'],
            framework_version=estimator_cfg['framework_version'],
            py_version=estimator_cfg['py_version'],
            instance_type=estimator_cfg['instance_type'],
            instance_count=estimator_cfg['instance_count'],
            sagemaker_session=estimator_cfg['sagemaker_session'],
            output_path=estimator_cfg['output_path'],
            hyperparameters=param_set
        )
    
    def fit(self):
        self.estimator.fit()

In [34]:
trainer = RandomForestCLFEstimator(
    estimator_cfg=ESTIMATOR_CFG,
    param_set=PARAMETERS
)

In [35]:
trainer.fit()

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2025-08-23-04-54-34-586
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' found

time="2025-08-23T13:54:35+09:00" level=warning msg="/private/var/folders/5z/57z97fnx0bsg5tc6bbg_p8s00000gn/T/tmp42p129_v/docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
time="2025-08-23T13:54:35+09:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmp42p129_v\".\nSet `external: true` to use an existing network"
 Container 96lt655uvd-algo-1-kwops  Creating
 algo-1-kwops The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested 
 Container 96lt655uvd-algo-1-kwops  Created
Attaching to 96lt655uvd-algo-1-kwops
96lt655uvd-algo-1-kwops  | /miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 

INFO:sagemaker.local.image:===== Job Complete =====
